## Making init files:

- need air temperature

- need rivers deltat

- need both boundary conditions. 

- need to interpolate deltat's onto our depth grid

In [2]:
## 
import arrow
import netCDF4 as nc
import glob
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy

## atmospheric files

downloaded from:
/home/tjarnik/projects/def-allen/SalishSea/forcing/atmospheric/GEM2.5/operational/FOR_TEPLO

In [11]:
tair_delta = 2.12

print(330.97263/(292.75018**4)*(292.75018+tair_delta)**4)

340.66445180697116


In [9]:
w = nc.Dataset('./FOR_TEPLO/ops_y2017m09d04.nc')
#print(w)
print(w['tair'][0,100,100])
print(w['therm_rad'])

TOLD = w['tair'][:]
LWCOLD = w['therm_rad'][:]
TNEW = TOLD+tair_delta

LWCNEW = LWCOLD/(TOLD**4)*(TNEW**4)

print(LWCOLD[0,100,100])
print(LWCNEW[0,100,100])

## LwcNEW/TNEW^4 = LwcOLD/TOLD^4 

## LwcNEW = LwcOLD/(TOLD^4) * (TNEW^4)

292.75018
<class 'netCDF4._netCDF4.Variable'>
float32 therm_rad(time_counter, y, x)
    _FillValue: 9.999e+20
    short_name: DLWRF_surface
    long_name: Downward Long-Wave Rad. Flux
    level: surface
    units: W/m^2
    coordinates: longitude latitude
unlimited dimensions: time_counter
current shape = (24, 266, 256)
filling on
330.97263
340.66443


In [14]:
#JDF_delta_ssc[i,:]
#JS_delta_ssc[i,:]

start ='2017-01-01'
end ='2017-12-31'

start_run = arrow.get(start)
end_run = arrow.get(end)

arrow_array = []

for r in arrow.Arrow.span_range('day', start_run, end_run):
    arrow_array.append(r)

In [19]:
for i in range(0,365):

    if i%30 == 0:
        print(i)
    
    tdate = arrow_array[i][0]
    print(tdate)
    yy = tdate.format('YYYY')
    mm = tdate.format('MM')
    dd = tdate.format('DD')
    ymd = f'y{yy}m{mm}d{dd}'
    
    #input file
    test_LO = nc.Dataset(f'./FOR_TEPLO/ops_{ymd}.nc')
    #print(test_LO)
    #input file for preindustrial DIC
    dsin = test_LO 
    
    TOLD = dsin['tair'][:]
    LWCOLD = dsin['therm_rad'][:]
    TNEW = TOLD+tair_delta

    LWCNEW = LWCOLD/(TOLD**4)*(TNEW**4)

    #output filename
    ncname = f'./adjusted_BCS/ops_TEPLO50_SSP245_{ymd}.nc'

    dsout = nc.Dataset(ncname, "w", format="NETCDF4")
    #Copy dimensions
    for dname, the_dim in dsin.dimensions.items():
        #print(dname)
        #print(len(the_dim))
        dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
        
    # Copy variables
    for v_name, varin in dsin.variables.items():
        #print(v_name)
        #if (v_name != 'votemper'):
        outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
        # Copy variable attributes
        outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})

        outVar[:] = varin[:]
    
    # copy new temp variables
    
    dsout['tair'][:] = TNEW
    dsout['therm_rad'][:] = LWCNEW

    dsout.close()
    

0
2017-01-01T00:00:00+00:00
2017-01-02T00:00:00+00:00
2017-01-03T00:00:00+00:00
2017-01-04T00:00:00+00:00
2017-01-05T00:00:00+00:00
2017-01-06T00:00:00+00:00
2017-01-07T00:00:00+00:00
2017-01-08T00:00:00+00:00
2017-01-09T00:00:00+00:00
2017-01-10T00:00:00+00:00
2017-01-11T00:00:00+00:00
2017-01-12T00:00:00+00:00
2017-01-13T00:00:00+00:00
2017-01-14T00:00:00+00:00
2017-01-15T00:00:00+00:00
2017-01-16T00:00:00+00:00
2017-01-17T00:00:00+00:00
2017-01-18T00:00:00+00:00
2017-01-19T00:00:00+00:00
2017-01-20T00:00:00+00:00
2017-01-21T00:00:00+00:00
2017-01-22T00:00:00+00:00
2017-01-23T00:00:00+00:00
2017-01-24T00:00:00+00:00
2017-01-25T00:00:00+00:00
2017-01-26T00:00:00+00:00
2017-01-27T00:00:00+00:00
2017-01-28T00:00:00+00:00
2017-01-29T00:00:00+00:00
2017-01-30T00:00:00+00:00
30
2017-01-31T00:00:00+00:00
2017-02-01T00:00:00+00:00
2017-02-02T00:00:00+00:00
2017-02-03T00:00:00+00:00
2017-02-04T00:00:00+00:00
2017-02-05T00:00:00+00:00
2017-02-06T00:00:00+00:00
2017-02-07T00:00:00+00:00
2017-02

2017-11-11T00:00:00+00:00
2017-11-12T00:00:00+00:00
2017-11-13T00:00:00+00:00
2017-11-14T00:00:00+00:00
2017-11-15T00:00:00+00:00
2017-11-16T00:00:00+00:00
2017-11-17T00:00:00+00:00
2017-11-18T00:00:00+00:00
2017-11-19T00:00:00+00:00
2017-11-20T00:00:00+00:00
2017-11-21T00:00:00+00:00
2017-11-22T00:00:00+00:00
2017-11-23T00:00:00+00:00
2017-11-24T00:00:00+00:00
2017-11-25T00:00:00+00:00
2017-11-26T00:00:00+00:00
330
2017-11-27T00:00:00+00:00
2017-11-28T00:00:00+00:00
2017-11-29T00:00:00+00:00
2017-11-30T00:00:00+00:00
2017-12-01T00:00:00+00:00
2017-12-02T00:00:00+00:00
2017-12-03T00:00:00+00:00
2017-12-04T00:00:00+00:00
2017-12-05T00:00:00+00:00
2017-12-06T00:00:00+00:00
2017-12-07T00:00:00+00:00
2017-12-08T00:00:00+00:00
2017-12-09T00:00:00+00:00
2017-12-10T00:00:00+00:00
2017-12-11T00:00:00+00:00
2017-12-12T00:00:00+00:00
2017-12-13T00:00:00+00:00
2017-12-14T00:00:00+00:00
2017-12-15T00:00:00+00:00
2017-12-16T00:00:00+00:00
2017-12-17T00:00:00+00:00
2017-12-18T00:00:00+00:00
2017-12-

In [5]:
def dilley_unsworth(lat, rpn_vp_on_ops, rpn_NT_on_ops, rpn_TT_on_ops, sigma):
    
    # rpn_TT_on_ops >> temperature in kelvin
    # 
# https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2008WR007394
# Dilley clear sky algorithm
    eo = rpn_vp_on_ops/1000.
    To = rpn_TT_on_ops
    w = 465*eo/To
    Lclr = 59.38 + 113.7*(To/273.16)**6 + 96.96*np.sqrt(w/2.5) # Dilley
    eclr = Lclr/(sigma * To**4)

    # Unsworth
    cf = rpn_NT_on_ops
    ewc = (1 - 0.84*cf)*eclr + 0.84*cf
#
    Lwc = ewc * sigma * To**4
    return Lwc